# Face detection demo
This notebook contains development of face detection and age determination including the live acquisition demo.

## Initialization

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import ipywidgets as ipw
import asyncio
from deepface import DeepFace
from PIL import Image
import io
import pprint
import os

## Main acquisition and proccessing code

In [4]:
# Main part of the code in a class
class MainClass:
    '''MainClass takes care of image acquisition, display and proccessing. 
    Additionally, it writes the status info into a textbox widget. 
    Acquisition is started by calling runAcquisition and stopped by calling
    stopAcquisition. It quits gracefully.'''
    def __init__(self):
        # Properties storing the data needed by the class
        self.cap = None
        self.stopRequested = Falsen
        self.analysisRequested = False
        self.image = None
        self.text = None
        # Parameters of the output encoded image
        self.encode_parameters = [int(cv2.IMWRITE_JPEG_QUALITY),75]
        # Which cascade to use
        self.xmlpath = './haarcascade_frontalface_default.xml'
        self.AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
        
        # Age detection using CV2
        prototxtPath = os.path.sep.join(["./models/deploy_age.prototxt"])
        weightsPath = os.path.sep.join(["./models/age_net.caffemodel"])
        self.ageNet = cv2.dnn.readNet(prototxtPath, weightsPath)
        
    async def run(self):
        '''Main routine which acquires image, proccessess it and displays it.'''
        # First, create the video capture object
        self.cap = cv2.VideoCapture('/dev/video0')
        
        i = 0
        while(True):
            i = i + 1
            # Acquire a new frame
            ret, frame = self.cap.read()
            
            # Proccess the image
            #self.detectAndAnalyzeDF(frame);
            img = self.detectFaces(frame);
            
            # Display the frame in a widget
            self.image.value = cv2.imencode('.jpg', img, self.encode_parameters)[1].tobytes()
            
            # Update status box
            #self.text.value = "Image {:0>5d} acquired!\n".format(i)
            
            # Wait for 0 time to release control to the main loop
            await asyncio.sleep(0)
            
            # Break the infinite loop if stop is requested
            if self.stopRequested:
                break
                
            if self.analysisRequested:
                self.detectAndAnalyzeDF(frame)
        
        # At the end, release the capture object
        self.cap.release()
        struct
        
    def runAcquisition(self):
        # Reset stop request
        self.stopRequested = False
        # Create a task and use it to run main routine async
        asyncio.create_task(self.run())

    def stopAcquisition(self):
        # Request a stop politely
        self.stopRequested = True
        
    def detectFaces(self, img):
        face_cascade = cv2.CascadeClassifier(self.xmlpath)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 6)
        for (x, y, w, h) in faces:
            # This is detection using on OpenCV dnn module
            face = img[x:x+w,y:y+h]
            faceBlob = cv2.dnn.blobFromImage(face,1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
            self.ageNet.setInput(faceBlob)
            preds = self.ageNet.forward()
            i = preds[0].argmax()
            age = self.AGE_BUCKETS[i]
            ageConfidence = preds[0][i]
    
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            lbly = y - 10 if y - 10 > 10 else y + 10
            text = "{}: {:.2f}%".format(age, ageConfidence * 100)
            cv2.putText(img, text, (x, lbly),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        return img
        
    def requestAnalysis(self):
        self.analysisRequested = True
        
    def detectAndAnalyzeDF(self,frame):
        img = np.asarray(frame)
        try:
            obj = DeepFace.analyze(img, actions = ['age', 'gender', 'race', 'emotion'])
            self.text.value = pprint.pformat(obj)
            self.analysisRequested = False
        except:
            self.text.value = "Face was not detected, move slightly!"
        

## GUI and control

In [3]:
# Build a simple GUI view
file = open("initial.jpg", "rb")
image = file.read()
file.close()
imgbox = ipw.Image(value=image,format='jpg', width='50%')
output = ipw.Textarea(layout=ipw.Layout(width='98%',height='300px'), disabled=True)
startBtn = ipw.Button(description='Start')
stopBtn = ipw.Button(description='Stop', disabled=True)
analyzeBtn = ipw.Button(description='Analyze', disabled=True)

# Create an instance of the main class
acquisition = MainClass()
# Assign output fields in the view
acquisition.image = imgbox
acquisition.text = output
    
# Callbacks definition
def on_startBtn_click(btn):
    startBtn.disabled = True
    stopBtn.disabled = False
    analyzeBtn.disabled = False
    acquisition.runAcquisition()

def on_stopBtn_click(btn):
    stopBtn.disabled = True
    analyzeBtn.disabled = True
    startBtn.disabled = False
    acquisition.stopAcquisition()
    
def on_analyzeBtn_click(btn):
    acquisition.requestAnalysis()

# Assign callbacks to buttons
startBtn.on_click(on_startBtn_click)
stopBtn.on_click(on_stopBtn_click)
analyzeBtn.on_click(on_analyzeBtn_click)

# Build and display the GUI window
buttons = ipw.Box([startBtn,stopBtn,analyzeBtn], layout=ipw.Layout(display='flex',
                                                        flex_flow='row',
                                                        justify_content='center',
                                                        align_items='center',
                                                        width='100%',
                                                        height='100px'))
controls = ipw.VBox([buttons,output], layout=ipw.Layout(width='50%',
                                                        justify_content='flex-end'))
window = ipw.HBox([imgbox, controls], layout=ipw.Layout(border='solid thin', 
                                                        width='100%', 
                                                        height='auto'))
# Display the GUI
display(window)

# Offline image age determination

## Includes

In [8]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import ipywidgets as ipw
import asyncio
from deepface import DeepFace
from PIL import Image
import io
import pprint
import os

## DeepFace

In [11]:
obj = DeepFace.analyze('Image-1.png', actions = ['age', 'gender', 'race', 'emotion'])
print()
pprint.pprint(obj)

Action: emotion: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.50it/s]


{'age': 42,
 'dominant_emotion': 'happy',
 'dominant_race': 'white',
 'emotion': {'angry': 39.19900526988569,
             'disgust': 0.0002931245839321723,
             'fear': 0.5934341586749743,
             'happy': 43.17440905265916,
             'neutral': 10.586657892067182,
             'sad': 0.7231420768072697,
             'surprise': 5.72306842780929},
 'gender': 'Man',
 'race': {'asian': 0.5643481388688087,
          'black': 0.14442461542785168,
          'indian': 3.125517815351486,
          'latino hispanic': 12.56120651960373,
          'middle eastern': 29.247862100601196,
          'white': 54.356640577316284},
 'region': {'h': 81, 'w': 81, 'x': 145, 'y': 62}}


## OpenCV

In [15]:
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]

prototxtPath = os.path.sep.join(["./models/deploy_age.prototxt"])
weightsPath = os.path.sep.join(["./models/age_net.caffemodel"])
ageNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load an image
face = cv2.imread("Image-1.png")
faceBlob = cv2.dnn.blobFromImage(face,1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
# make predictions on the age and find the age bucket with
# the largest corresponding probability
ageNet.setInput(faceBlob)
preds = ageNet.forward()
i = preds[0].argmax()
age = AGE_BUCKETS[i]
ageConfidence = preds[0][i]
# display the predicted age to our terminal
text = "{}: {:.2f}%".format(age, ageConfidence * 100)
print("[INFO] {}".format(text))

[INFO] (60-100): 99.32%
